# Combine hourly matched dataFrames of metrological data with Central Node and LoRa Nodes. 

In [1]:
using CSV
using DataFrames
using Dates
using Statistics

In [76]:
# Local machine path to UTC hourly PM dataFrames
df53_h = CSV.read("/Users/prabu/Desktop/Joppa/UTC_df/hourly_avg_df/df53_hourly_avg.csv", DataFrame);
df54_h = CSV.read("/Users/prabu/Desktop/Joppa/UTC_df/hourly_avg_df/df54_hourly_avg.csv", DataFrame);
dfCent_h = CSV.read("/Users/prabu/Desktop/Joppa/UTC_df/hourly_avg_df/Cent_hourly_avg.csv", DataFrame);

# Local machine path to matched dataFrames
df53 = CSV.read("/Users/prabu/Desktop/Joppa/ML_Data/match_df/L53_weather_match.csv", DataFrame);
df54 = CSV.read("/Users/prabu/Desktop/Joppa/ML_Data/match_df/L54_weather_match.csv", DataFrame);
dfCent = CSV.read("/Users/prabu/Desktop/Joppa/ML_Data/match_df/Cent_weather_match.csv", DataFrame);

In [77]:
# Convert to dateTime format
df53.dateTime = DateTime.(df53.dateTime,"yyyy-mm-dd HH:MM:SS") 
df54.dateTime = DateTime.(df54.dateTime,"yyyy-mm-dd HH:MM:SS") 
dfCent.dateTime = DateTime.(dfCent.dateTime,"yyyy-mm-dd HH:MM:SS") ;

In [78]:
df53_com = outerjoin(df53_h, df53, on = :dateTime, makeunique=true)
df54_com = outerjoin(df54_h, df54, on = :dateTime, makeunique=true)
dfCent_com = outerjoin(dfCent_h, dfCent, on = :dateTime, makeunique=true);

In [20]:
CSV.write("/Users/prabu/Desktop/Joppa/ML_Data/match_df/L53_PM_weather.csv", df53_com)
CSV.write("/Users/prabu/Desktop/Joppa/ML_Data/match_df/L54_PM_weather.csv", df54_com)
CSV.write("/Users/prabu/Desktop/Joppa/ML_Data/match_df/Cent_PM_weather.csv", dfCent_com)

"/Users/prabu/Desktop/Joppa/ML_Data/match_df/Cent_PM_weather.csv"

In [79]:
# Removing extra columns values
select!(df53_com, Not(:GPGGALR_Latitude_mean))
select!(df53_com, Not(:GPGGALR_Longitude_mean))
select!(df54_com, Not(:GPGGALR_Latitude_mean))
select!(df54_com, Not(:GPGGALR_Longitude_mean))

# Rename
rename!(df53_com,:GPGGALR_Altitude_mean => :Altitude)
rename!(df54_com,:GPGGALR_Altitude_mean => :Altitude)
rename!(dfCent_com,:BME280_altitude_mean => :Altitude)

rename!(dfCent_com,:BME280_temperature_mean => :BME280_Temperature_mean)
rename!(dfCent_com,:BME280_pressure_mean => :BME280_Pressure_mean)
rename!(dfCent_com,:BME280_humidity_mean => :BME280_Humidity_mean)

# Removing missing values
df53_com = df53_com[completecases(df53_com), :]
df54_com = df54_com[completecases(df54_com), :]
dfCent_com = dfCent_com[completecases(dfCent_com), :]

# Combine all the dataFrames vertically
df = outerjoin(df53_com, df54_com, dfCent_com, on = intersect(names(df53_com), names(df54_com), names(dfCent_com)))

# Saving
CSV.write("/Users/prabu/Desktop/Joppa/ML_Data/match_df/Combined_PM_weather.csv", df)

"/Users/prabu/Desktop/Joppa/ML_Data/match_df/Combined_PM_weather.csv"